In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from function import *
from function_preparation import *
from ClassModel import LSTM_upgrade, ModelManagerGPT

ImportError: cannot import name 'LSTM_upgrade' from 'ClassModel' (e:\MyPythonCode\Crypto\notebooks\ClassModel.py)

1. Data Preparation

1.1 Đọc df raw

In [2]:
eth_path = r"E:\MyPythonCode\Crypto\data\csv\binance_ETHUSDT.csv"
df = pd.read_csv(eth_path)
df

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,zero
0,1695484800000,1590.48,1594.15,1588.85,1593.42,3396.4181,1695488399999,5.402303e+06,10095,2020.9590,3.214783e+06,0
1,1695488400000,1593.43,1594.33,1590.21,1591.23,3497.1178,1695491999999,5.566259e+06,9236,1800.6868,2.865973e+06,0
2,1695492000000,1591.23,1595.25,1590.96,1592.89,3114.2469,1695495599999,4.962522e+06,8528,1393.8469,2.220929e+06,0
3,1695495600000,1592.88,1595.31,1592.88,1594.79,1551.8648,1695499199999,2.473972e+06,7482,766.3382,1.221626e+06,0
4,1695499200000,1594.79,1595.46,1593.10,1593.18,1656.6078,1695502799999,2.640695e+06,6634,764.3698,1.218440e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1727002800000,2580.15,2584.77,2572.05,2580.59,7885.5562,1727006399999,2.035016e+07,61939,4021.3146,1.037863e+07,0
8756,1727006400000,2580.59,2585.53,2575.12,2581.17,4044.7767,1727009999999,1.043823e+07,45180,2112.3441,5.451533e+06,0
8757,1727010000000,2581.17,2588.00,2566.19,2570.70,12832.4517,1727013599999,3.304580e+07,86826,4149.7031,1.068716e+07,0
8758,1727013600000,2570.70,2574.60,2554.82,2564.04,13095.4564,1727017199999,3.358454e+07,130807,5677.9491,1.456298e+07,0


1.2 Xử lý sơ bộ thành df_resampled (nhưng lười sửa nên đặt thẳng thành df luôn)

In [3]:
df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
df = df.resample('1h', on='Open Time').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Quote Volume': 'sum',
    'Number of Trades': 'sum',
    'Taker Buy Base Volume': 'sum',
    'Taker Buy Quote Volume': 'sum',
}).dropna()
df

,Open,High,Low,Close,Volume,Quote Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume
Open Time,,,,,,,,,
2023-09-23 16:00:00,1590.48,1594.15,1588.85,1593.42,3396.4181,5.402303e+06,10095,2020.9590,3.214783e+06
2023-09-23 17:00:00,1593.43,1594.33,1590.21,1591.23,3497.1178,5.566259e+06,9236,1800.6868,2.865973e+06
2023-09-23 18:00:00,1591.23,1595.25,1590.96,1592.89,3114.2469,4.962522e+06,8528,1393.8469,2.220929e+06
2023-09-23 19:00:00,1592.88,1595.31,1592.88,1594.79,1551.8648,2.473972e+06,7482,766.3382,1.221626e+06
2023-09-23 20:00:00,1594.79,1595.46,1593.10,1593.18,1656.6078,2.640695e+06,6634,764.3698,1.218440e+06
...,...,...,...,...,...,...,...,...,...
2024-09-22 11:00:00,2580.15,2584.77,2572.05,2580.59,7885.5562,2.035016e+07,61939,4021.3146,1.037863e+07
2024-09-22 12:00:00,2580.59,2585.53,2575.12,2581.17,4044.7767,1.043823e+07,45180,2112.3441,5.451533e+06
2024-09-22 13:00:00,2581.17,2588.00,2566.19,2570.70,12832.4517,3.304580e+07,86826,4149.7031,1.068716e+07


1.3.1 Chuẩn bị dữ liệu [biến X]

# 1. Tính toán 12 chỉ báo kỹ thuật
Tổng kết gợi ý cho period:
- SMA/EMA: 5 hoặc 10
- RSI: 9 hoặc 14
- MACD: 12, 26, 9 (mặc định)
- Bollinger Bands: 10 đến 20
- ATR: 10 hoặc 14
- Stochastic Oscillator: 9 hoặc 14
- OBV: giữ nguyên
- ROC: 9 hoặc 12
- CMF: 10 hoặc 20
- ADX: 10 hoặc 14
- SAR: giữ nguyên acceleration 0.02 và maximum 0

In [4]:
df['SMA_10'] = calculate_sma(df, period=10)
df['EMA_10'] = calculate_ema(df, period=10)
df['RSI_14'] = calculate_rsi(df, period=14)
df['MACD'], df['Signal_Line'] = calculate_macd(df)
df['Bollinger_Upper'], df['Bollinger_Lower'] = calculate_bollinger_bands(df, period=20)
df['ATR'] = calculate_atr(df, period=14)
df['Stochastic_Oscillator'] = calculate_stochastic_oscillator(df, period=14)
df['OBV'] = calculate_obv(df)
df['ROC'] = calculate_roc(df, period=12)
df['CMF'] = calculate_cmf(df, period=20)
df['ADX_14'] = calculate_adx(df, period=14)
df['SAR'] = calculate_sar(df, acceleration=0.02, maximum=0.2)
df


,Open,High,Low,Close,Volume,Quote Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,SMA_10,...,-DM,TR,+DM_Smoothed,-DM_Smoothed,TR_Smoothed,+DI,-DI,DX,ADX_14,SAR
Open Time,,,,,,,,,,,,,,,,,,,,,
2023-09-23 16:00:00,1590.48,1594.15,1588.85,1593.42,3396.4181,5.402303e+06,10095,2020.9590,3.214783e+06,NaN,...,0.00,5.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-23 17:00:00,1593.43,1594.33,1590.21,1591.23,3497.1178,5.566259e+06,9236,1800.6868,2.865973e+06,NaN,...,0.00,4.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1588.850000
2023-09-23 18:00:00,1591.23,1595.25,1590.96,1592.89,3114.2469,4.962522e+06,8528,1393.8469,2.220929e+06,NaN,...,0.00,4.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1588.959600
2023-09-23 19:00:00,1592.88,1595.31,1592.88,1594.79,1551.8648,2.473972e+06,7482,766.3382,1.221626e+06,NaN,...,0.00,2.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1589.211216
2023-09-23 20:00:00,1594.79,1595.46,1593.10,1593.18,1656.6078,2.640695e+06,6634,764.3698,1.218440e+06,NaN,...,0.00,2.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1589.577143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-22 11:00:00,2580.15,2584.77,2572.05,2580.59,7885.5562,2.035016e+07,61939,4021.3146,1.037863e+07,2587.397,...,0.00,12.72,6.195714,3.645714,18.799286,32.957179,19.392834,25.910872,39.036835,2628.915344
2024-09-22 12:00:00,2580.59,2585.53,2575.12,2581.17,4044.7767,1.043823e+07,45180,2112.3441,5.451533e+06,2586.043,...,0.00,10.41,5.599286,3.645714,18.385714,30.454545,19.829060,21.131113,38.387942,2626.543130
2024-09-22 13:00:00,2581.17,2588.00,2566.19,2570.70,12832.4517,3.304580e+07,86826,4149.7031,1.068716e+07,2583.642,...,8.93,21.81,2.232143,4.283571,16.394286,13.615371,26.128442,31.484324,36.243827,2624.265805


# 2. Dropna và xoá hẳn 20 hàng đầu tiên thiếu dữ liệu

In [5]:
df = df.dropna()
df = df.iloc[20:].reset_index(drop=True)
df

,Open,High,Low,Close,Volume,Quote Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,SMA_10,...,-DM,TR,+DM_Smoothed,-DM_Smoothed,TR_Smoothed,+DI,-DI,DX,ADX_14,SAR
0,1576.70,1577.00,1573.67,1576.03,5061.7066,7.973899e+06,15379,2692.3562,4.241202e+06,1575.486,...,0.00,3.33,1.094286,1.240714,6.518571,16.787201,19.033531,6.271031,42.715306,1578.710862
1,1576.03,1589.74,1576.03,1584.64,15733.8816,2.493128e+07,30653,8215.4748,1.301425e+07,1576.313,...,0.00,13.71,1.501429,1.240714,6.573571,22.840378,18.874280,9.507684,40.667456,1563.010000
2,1584.64,1597.50,1584.63,1590.61,17918.3936,2.853673e+07,28680,9124.4044,1.453009e+07,1577.517,...,0.00,12.87,2.055714,1.232857,7.045714,29.176805,17.497972,25.021720,39.552974,1563.544600
3,1590.61,1595.08,1588.08,1592.46,6730.2225,1.070817e+07,15721,3277.7818,5.214535e+06,1578.638,...,0.00,7.00,2.000000,1.232857,7.215714,27.717284,17.085726,23.729563,38.396797,1564.902816
4,1592.46,1594.00,1590.85,1591.28,3590.0051,5.716626e+06,9593,1529.7614,2.435824e+06,1579.927,...,0.00,3.15,2.000000,1.067857,7.057857,28.337213,15.130048,30.384168,37.555687,1566.206703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8709,2580.15,2584.77,2572.05,2580.59,7885.5562,2.035016e+07,61939,4021.3146,1.037863e+07,2587.397,...,0.00,12.72,6.195714,3.645714,18.799286,32.957179,19.392834,25.910872,39.036835,2628.915344
8710,2580.59,2585.53,2575.12,2581.17,4044.7767,1.043823e+07,45180,2112.3441,5.451533e+06,2586.043,...,0.00,10.41,5.599286,3.645714,18.385714,30.454545,19.829060,21.131113,38.387942,2626.543130
8711,2581.17,2588.00,2566.19,2570.70,12832.4517,3.304580e+07,86826,4149.7031,1.068716e+07,2583.642,...,8.93,21.81,2.232143,4.283571,16.394286,13.615371,26.128442,31.484324,36.243827,2624.265805
8712,2570.70,2574.60,2554.82,2564.04,13095.4564,3.358454e+07,130807,5677.9491,1.456298e+07,2580.184,...,11.37,19.78,1.572857,5.095714,15.827857,9.937272,32.194594,52.827763,35.297201,2620.781257


1.3.2 Chuẩn bị dữ liệu [biến y]

- Nếu [hàng tiếp theo] giá cao hơn [hàng hiện tại] : Label = 1
- Nếu [hàng tiếp theo] giá thấp hơn [hàng hiện tại]: label = 0

In [6]:
df['Label'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df

,Open,High,Low,Close,Volume,Quote Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,SMA_10,...,TR,+DM_Smoothed,-DM_Smoothed,TR_Smoothed,+DI,-DI,DX,ADX_14,SAR,Label
0,1576.70,1577.00,1573.67,1576.03,5061.7066,7.973899e+06,15379,2692.3562,4.241202e+06,1575.486,...,3.33,1.094286,1.240714,6.518571,16.787201,19.033531,6.271031,42.715306,1578.710862,1
1,1576.03,1589.74,1576.03,1584.64,15733.8816,2.493128e+07,30653,8215.4748,1.301425e+07,1576.313,...,13.71,1.501429,1.240714,6.573571,22.840378,18.874280,9.507684,40.667456,1563.010000,1
2,1584.64,1597.50,1584.63,1590.61,17918.3936,2.853673e+07,28680,9124.4044,1.453009e+07,1577.517,...,12.87,2.055714,1.232857,7.045714,29.176805,17.497972,25.021720,39.552974,1563.544600,1
3,1590.61,1595.08,1588.08,1592.46,6730.2225,1.070817e+07,15721,3277.7818,5.214535e+06,1578.638,...,7.00,2.000000,1.232857,7.215714,27.717284,17.085726,23.729563,38.396797,1564.902816,0
4,1592.46,1594.00,1590.85,1591.28,3590.0051,5.716626e+06,9593,1529.7614,2.435824e+06,1579.927,...,3.15,2.000000,1.067857,7.057857,28.337213,15.130048,30.384168,37.555687,1566.206703,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8709,2580.15,2584.77,2572.05,2580.59,7885.5562,2.035016e+07,61939,4021.3146,1.037863e+07,2587.397,...,12.72,6.195714,3.645714,18.799286,32.957179,19.392834,25.910872,39.036835,2628.915344,1
8710,2580.59,2585.53,2575.12,2581.17,4044.7767,1.043823e+07,45180,2112.3441,5.451533e+06,2586.043,...,10.41,5.599286,3.645714,18.385714,30.454545,19.829060,21.131113,38.387942,2626.543130,0
8711,2581.17,2588.00,2566.19,2570.70,12832.4517,3.304580e+07,86826,4149.7031,1.068716e+07,2583.642,...,21.81,2.232143,4.283571,16.394286,13.615371,26.128442,31.484324,36.243827,2624.265805,0
8712,2570.70,2574.60,2554.82,2564.04,13095.4564,3.358454e+07,130807,5677.9491,1.456298e+07,2580.184,...,19.78,1.572857,5.095714,15.827857,9.937272,32.194594,52.827763,35.297201,2620.781257,1


1.4 Tạo sequence X,y

In [7]:
def create_sequences_in_numpy(df, timesteps):
    X = []
    y = []

    for i in range(len(df) - timesteps):
        # Giống như việc quét chọn 1 khu vực trong bảng tính, chỉ định index từ đâu đến đâu    

        # Lấy các cột đặc trưng làm X
        X.append(df.iloc[i:i+timesteps, :-1].values)  # Tất cả cột trừ cột 'Label' là đầu vào
        # *** Giải thích ***
        # [tức là lấy hàng từ (i) đến (i+timesteps)-1, và cột giá trị là :-1] (lấy all từ cột cuối)
        
        # Lấy cột 'Label' ở vị trí i+timesteps làm y
        y.append(df.iloc[i+timesteps, -1])  # Cột 'Label' là đầu ra
        # *** Giải thích ***
        # [tức là lấy hàng (i+timesteps), và cột giá trị là -1] (lấy cột cuối)

    return np.array(X), np.array(y)

# Ví dụ với timesteps là 20
timesteps = 24
X, y = create_sequences_in_numpy(df, timesteps)

# Kiểm tra kích thước của X và y
print("X shape:", X.shape)  # (số lượng mẫu, timesteps, số đặc trưng)
print("y shape:", y.shape)  # (số lượng mẫu,)


X shape: (8690, 24, 34)
y shape: (8690,)


In [8]:
# Chọn một cặp thử (ví dụ, cặp đầu tiên)
example_idx = 2

# In thử X và y tại index được chọn
print("X[{}]:\n".format(example_idx), X[example_idx])
print("\ny[{}]:\n".format(example_idx), y[example_idx])


X[2]:
 [[ 1.58464000e+03  1.59750000e+03  1.58463000e+03  1.59061000e+03
   1.79183936e+04  2.85367334e+07  2.86800000e+04  9.12440440e+03
   1.45300919e+07  1.57751700e+03  1.57980522e+03  6.09205021e+01
  -2.03798748e+00 -3.65040136e+00  1.59084980e+03  1.56732820e+03
   7.04571429e+00  7.92156863e+01 -5.09851015e+04  7.55056408e-01
   1.69024581e-01  7.76000000e+00 -8.60000000e+00  7.76000000e+00
   0.00000000e+00  1.28700000e+01  2.05571429e+00  1.23285714e+00
   7.04571429e+00  2.91768045e+01  1.74979724e+01  2.50217202e+01
   3.95529739e+01  1.56354460e+03]
 [ 1.59061000e+03  1.59508000e+03  1.58808000e+03  1.59246000e+03
   6.73022250e+03  1.07081698e+07  1.57210000e+04  3.27778180e+03
   5.21453503e+06  1.57863800e+03  1.58210609e+03  6.56144068e+01
  -8.68109586e-01 -3.09394300e+00  1.59150961e+03  1.56692139e+03
   7.21571429e+00  8.47963801e+01 -4.42548790e+04  1.02069946e+00
   1.66397566e-01 -2.42000000e+00 -3.45000000e+00  0.00000000e+00
   0.00000000e+00  7.00000000e+00 

1.5 Chuyển đổi X, y thành Tensor

In [9]:
# Chuyển đổi X và y thành Tensor
X_tensor = torch.tensor(X, dtype=torch.float32)  # Đầu vào X (kích thước [samples, timesteps, features])
y_tensor = torch.tensor(y, dtype=torch.float32)  # Nhãn y (kích thước [samples,])


1.6 Tạo Dataset và chia dữ liệu

In [10]:
# Tạo Dataset
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

1.7 Tạo DataLoader

In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

1.8 [3] Các bước check đơn giản

In [12]:
check_dataset_splits(dataset, train_dataset, val_dataset, test_dataset)
# check_batch_in_loader(train_loader) # print ra khá dài
# check_unique_labels_in_dataset(y_tensor)

Check1: Kiểm tra số lượng mẫu trong train_dataset, val_dataset và test_dataset (nếu có)
Số mẫu trong tập huấn luyện: 6083
Số mẫu trong tập validation: 1303
Số mẫu trong tập kiểm thử: 1304
---------------------------------------------------------------


1.9 [2] Các bước check nâng cao

In [13]:
check_label_distribution(train_dataset, val_dataset, test_dataset)
# check_dataloader_functionality(train_loader, test_loader)

Check4: Kiểm tra phân phối nhãn giữa train/val/test
Phân phối nhãn trong tập huấn luyện: [2977 3106]
Phân phối nhãn trong tập validation: [632 671]
Phân phối nhãn trong tập kiểm thử: [631 673]
---------------------------------------------------------------


2. Model Preparation

2.1 Tham số

In [14]:
input_size = X.shape[2]  # Số đặc trưng đầu vào
hidden_size = 64  # Số lượng đơn vị ẩn trong LSTM
output_size = 1  # Số lớp đầu ra (cho bài toán hồi quy)
num_layers = 2  # Số lớp LSTM
ahead = 1  # Số bước dự đoán phía trước (thường là 1)

In [15]:
print("Model Parameters---------------------------------")
print("Số lượng features (số cột) dùng làm input: ", input_size)
print(f"Danh sách nhãn: {torch.unique(y_tensor)}")
print(f"Số lượng nhãn duy nhất (output_size): {output_size}")
print("Số lượng đơn vị ẩn để xử lý thông tin (hidden_size): ", hidden_size)
print("Số lượng lớp LSTM cần sử dụng: ", num_layers)
print("Ahead trong trường hợp dự đoán nhãn: ", ahead)

Model Parameters---------------------------------
Số lượng features (số cột) dùng làm input:  34
Danh sách nhãn: tensor([0., 1.])
Số lượng nhãn duy nhất (output_size): 1
Số lượng đơn vị ẩn để xử lý thông tin (hidden_size):  64
Số lượng lớp LSTM cần sử dụng:  2
Ahead trong trường hợp dự đoán nhãn:  1


2.2 Khởi tạo

In [16]:
# Khởi tạo mô hình
model_lstm_regression = LSTM_upgrade(input_size=input_size, 
                    hidden_size=hidden_size, 
                    output_size=output_size,
                    num_layers=num_layers,
                    ahead=ahead, 
                    task_type='regression')  # task_type='regression' cho bài toán hồi quy
model_lstm_regression

LSTM_upgrade(
  (lstm): LSTM(34, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [17]:
# Khởi tạo mô hình
model_lstm_classification = LSTM_upgrade(input_size=input_size, 
                    hidden_size=hidden_size, 
                    output_size=output_size,
                    num_layers=num_layers,
                    ahead=ahead, 
                    task_type='classification')  # task_type='classification' cho bài toán phân loại
model_lstm_classification

LSTM_upgrade(
  (lstm): LSTM(34, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

2.3 Chạy thử model với một batch

# Khi nào chọn classification?
- khi bạn chỉ quan tâm đến [xu hướng] tăng hay giảm mà [không cần biết mức độ thay đổi].
- khi bài toán của bạn là [nhị phân rõ ràng] và bạn muốn sử dụng các mô hình được tối ưu cho phân loại.

In [18]:
# test với task Classification
test_model_on_batch(model_lstm_classification, train_loader)

-----------------------------------------------
Chạy thử model với 1 batch để xem output_shape
Kích thước của batch đầu vào X_batch: torch.Size([32, 24, 34])
Kích thước của nhãn y_batch: torch.Size([32])
Kích thước đầu ra của mô hình: torch.Size([32, 1])
Đầu ra của mô hình (outputs): tensor([[-0.0183],
        [-0.0026],
        [ 0.0298],
        [ 0.0317],
        [ 0.0600],
        [ 0.0258],
        [ 0.0491],
        [ 0.0473],
        [ 0.0280],
        [ 0.0450],
        [ 0.0668],
        [ 0.0144],
        [ 0.0620],
        [ 0.0072],
        [ 0.0241],
        [ 0.0094],
        [ 0.0044],
        [ 0.0373],
        [ 0.0472],
        [ 0.0515],
        [ 0.0270],
        [ 0.0267],
        [ 0.0252],
        [-0.0016],
        [ 0.0169],
        [ 0.0097],
        [ 0.0218],
        [ 0.0400],
        [-0.0060],
        [ 0.0146],
        [ 0.0195],
        [ 0.0548]])


# Khi nào chọn regression?
- khi bạn muốn dự đoán [mức độ thay đổi giá] chứ không chỉ xu hướng (tăng/giảm).
- khi bạn có thể chuyển kết quả hồi quy thành bài toán nhị phân (bằng cách áp dụng ngưỡng) hoặc khi biên độ thay đổi là thông tin quan trọng.

In [19]:
# test với task Regression
test_model_on_batch(model_lstm_regression, train_loader)

-----------------------------------------------
Chạy thử model với 1 batch để xem output_shape
Kích thước của batch đầu vào X_batch: torch.Size([32, 24, 34])
Kích thước của nhãn y_batch: torch.Size([32])
Kích thước đầu ra của mô hình: torch.Size([32, 1, 1])
Đầu ra của mô hình (outputs): tensor([[[ 0.0485]],

        [[ 0.0513]],

        [[ 0.0524]],

        [[ 0.0014]],

        [[ 0.0443]],

        [[ 0.0380]],

        [[ 0.0389]],

        [[ 0.0286]],

        [[-0.0010]],

        [[ 0.0358]],

        [[ 0.0430]],

        [[-0.0191]],

        [[-0.0186]],

        [[ 0.0371]],

        [[ 0.0378]],

        [[ 0.0548]],

        [[ 0.0293]],

        [[-0.0135]],

        [[ 0.0183]],

        [[ 0.0382]],

        [[-0.0164]],

        [[ 0.0476]],

        [[-0.0061]],

        [[ 0.0391]],

        [[ 0.0383]],

        [[ 0.0538]],

        [[ 0.0496]],

        [[ 0.0411]],

        [[ 0.0328]],

        [[ 0.0326]],

        [[ 0.0442]],

        [[ 0.0537]]])


3. Model_manager

3.1 Truyền model vào optimizer và scheduler

- optimizer truyền vào model_manager_GPT
- scheduler truyền vào hàm train

In [20]:
# 3.1.1 Option linh hoạt
# Tạo optimizer và ReduceLROnPlateau
# ReduceLROnPlateau: Giảm learning rate khi hiệu suất mô hình không cải thiện sau một số epoch nhất định, 
# ..giúp điều chỉnh linh hoạt hơn theo quá trình huấn luyện.
optimizer = optim.Adam(model_lstm_regression.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# 3.1.2 Option cố định
# Tạo optimizer và StepLR-scheduler
# StepLR: Giảm learning rate theo chu kỳ cố định sau một số epoch.
# optimizer = optim.Adam(model_lstm_regression.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

e:\MyPythonCode\Crypto\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


3.2 Khởi tạo model_manager_GPT (bản cải tiến)

3.2.1 Task Regression chọn 1 trong 2 hàm loss:
- torch.nn.MSELoss(): Hàm này nhạy cảm với outliers (các điểm dữ liệu có giá trị quá lớn hoặc quá nhỏ).  MSE, với khả năng phóng đại lỗi lớn, thường được sử dụng để đảm bảo mô hình sẽ cố gắng dự đoán chính xác hơn trong những trường hợp giá trị sai lệch quá lớn.
- torch.nn.L1Loss(): MAE ít nhạy cảm hơn với outliers so với MSE, phù hợp khi muốn đánh giá tổng quát lỗi mà không muốn bị ảnh hưởng quá nhiều bởi sai số lớn.
- thông thường dùng MSE nhiều hơn.

In [21]:
# 3.2.1 Task Regression dùng 
model_manager_regression = ModelManagerGPT(
    model=model_lstm_regression,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    lr=0.001,
    patience=10,
    criterion=torch.nn.MSELoss(),
    task_type='regression'
)

3.2.2 Task classification thì không cần chọn

In [22]:
# 3.2.1 Task Classification dùng 
model_manager_classification = ModelManagerGPT(
    model=model_lstm_regression,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    lr=0.001,
    patience=10,
    criterion=torch.nn.CrossEntropyLoss(),
    task_type='classification'
)

3.3 Train

In [32]:
model_manager_regression.train(num_epochs=50, save_dir='notebools//models', scheduler=scheduler)

e:\MyPythonCode\Crypto\.venv\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], time: 1s, loss: 0.2499, val_loss: 0.2498, 
Early stopping at epoch 1


e:\MyPythonCode\Crypto\.venv\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
e:\MyPythonCode\Crypto\.venv\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([23])) that is different to the input size (torch.Size([23, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


AttributeError: 'ModelManagerGPT' object has no attribute 'load_model'